In [20]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np

d1 = pd.read_csv("data/材料1.csv")
d2 = pd.read_csv("data/材料2.csv")
d3 = pd.read_csv("data/材料3.csv")
d4 = pd.read_csv("data/材料4.csv")

d1['磁芯材料'] = '材料1'
d2['磁芯材料'] = '材料2'
d3['磁芯材料'] = '材料3'
d4['磁芯材料'] = '材料4'

data = pd.concat([d1, d2, d3, d4])

# data = data.drop(data.columns[-1], axis=1)  # 拼接 data 为一整个数据集


magnetic_flux_density = data.iloc[:, 4:-1].values

std_flux = np.std(magnetic_flux_density, axis=1)  # 标准差
max_flux = np.max(magnetic_flux_density, axis=1)  # 峰值

data['标准差'] = std_flux
data['峰值'] = max_flux

pd.set_option('future.no_silent_downcasting', True)

data['励磁波形'] = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3})



# print(data.shape)
# print(data.head())

lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
lst.append('磁芯材料')
lst.append('标准差')
lst.append('峰值')
data.columns = lst


# to = pd.DataFrame(data)
# to.to_csv("~/Desktop/data.csv", index=False)

# filter_data = data[['温度', '频率', '励磁波形', '材料', '峰值', '磁芯损耗']]

data_updated = data[['温度', '励磁波形', '磁芯材料', '磁芯损耗']]


# 重命名列名以便于使用
data_updated.columns = ['Temperature', 'Excitation', 'Material', 'Core_Loss']


# 初始化OneHotEncoder，使用'drop="first"'避免多重共线性
encoder = OneHotEncoder(drop=None)
material_encoded = encoder.fit_transform(data_updated[['Material']]).toarray()

# 将编码后的结果转换为DataFrame并重置索引
material_encoded_df = pd.DataFrame(material_encoded, columns=encoder.get_feature_names_out(['Material'])).reset_index(drop=True)

# 原数据集重置索引以确保匹配
data_updated = data_updated.reset_index(drop=True)

# 将编码后的列与原数据框（去掉原来的'材料'列）合并
data_encoded = pd.concat([data_updated.drop('Material', axis=1), material_encoded_df], axis=1)

filter_data = data[['温度', '励磁波形', '磁芯材料', '磁芯损耗']]


data_encoded.head()

# df = pd.DataFrame(data_encoded)
# df.to_csv("~/Desktop/filter_data.csv", index=False)

,Temperature,Excitation,Core_Loss,Material_材料1,Material_材料2,Material_材料3,Material_材料4
0,25,1,1997.955250,1.0,0.0,0.0,0.0
1,25,1,2427.749830,1.0,0.0,0.0,0.0
2,25,1,3332.725760,1.0,0.0,0.0,0.0
3,25,1,4502.908007,1.0,0.0,0.0,0.0
4,25,1,6063.023248,1.0,0.0,0.0,0.0


In [37]:
# 归一化
lower_bound = 0.1

columns_to_normalize = ['温度', '频率', '励磁波形', '材料', '峰值']

data_normalized_v2 = filter_data.copy()
for column in columns_to_normalize:
    min_val = data[column].min()
    max_val = data[column].max()
    data_normalized_v2[column] = lower_bound + (data[column] - min_val) / (max_val - min_val) * (1 - lower_bound)

data_normalized_v2.head()


,温度,频率,励磁波形,材料,峰值,磁芯损耗
0,0.1,0.10008,1.0,0.1,0.156939,1997.955250
1,0.1,0.10006,1.0,0.1,0.164558,2427.749830
2,0.1,0.10006,1.0,0.1,0.176757,3332.725760
3,0.1,0.10006,1.0,0.1,0.190037,4502.908007
4,0.1,0.10008,1.0,0.1,0.204894,6063.023248


In [43]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# 
# # 定义特征和目标列
# features = ['温度', '频率', '励磁波形', '材料', '峰值']
# target = '磁芯损耗'
# 
# # 从归一化后的数据中提取特征和目标
# X = data_normalized_v2[features].values
# y = data_normalized_v2[target].values
# 
# # 定义多项式的阶数为 2
# poly_degree = 2
# 
# # 创建多项式特征转换器，用于将特征转换为多项式形式
# polynomial_features = PolynomialFeatures(degree=poly_degree, include_bias=False)
# 
# # 将特征转换为多项式特征
# X_poly = polynomial_features.fit_transform(X)
# 
# # 使用线性回归模型拟合多项式特征
# model = LinearRegression()
# model.fit(X_poly, y)
# 
# # 输出拟合后的模型参数（即多项式的系数）
# print("拟合后的多项式系数：", model.coef_)
# print("拟合后的截距：", model.intercept_)
# 
# # 输出特征名称及其对应的多项式系数
# feature_names = polynomial_features.get_feature_names_out(features)
# for i, coef in enumerate(model.coef_):
#     print(f"{feature_names[i]} 的系数为: {coef}")

拟合后的多项式系数： [ 4.70352906e+04 -1.37264253e+06 -6.65966112e+04 -1.84849244e+05
 -2.51870193e+06  1.07734412e+05 -1.57828864e+05  3.71768398e+03
 -8.48678145e+02 -6.33174263e+05  3.69785672e+05 -6.58681592e+04
  1.40766862e+05  7.00912107e+06  4.03087480e+04  4.65741281e+04
 -3.63438803e+04  8.68949259e+03  5.54385034e+05  2.61413195e+06]
拟合后的截距： 458871.20618448267
温度 的系数为: 47035.29061679917
频率 的系数为: -1372642.5274529574
励磁波形 的系数为: -66596.61121809878
材料 的系数为: -184849.2435697119
峰值 的系数为: -2518701.927310431
温度^2 的系数为: 107734.41150113427
温度 频率 的系数为: -157828.86394572284
温度 励磁波形 的系数为: 3717.6839847522556
温度 材料 的系数为: -848.6781445501582
温度 峰值 的系数为: -633174.2632215221
频率^2 的系数为: 369785.67202972533
频率 励磁波形 的系数为: -65868.15918005812
频率 材料 的系数为: 140766.8621911864
频率 峰值 的系数为: 7009121.073752275
励磁波形^2 的系数为: 40308.74802113532
励磁波形 材料 的系数为: 46574.12808066545
励磁波形 峰值 的系数为: -36343.88029595151
材料^2 的系数为: 8689.492592276767
材料 峰值 的系数为: 554385.0337640304
峰值^2 的系数为: 2614131.9534734767


In [48]:
# # 对系数进行归一化，使其在 0 和 1 之间
# coefficients = model.coef_
# normalized_coefficients = (coefficients - coefficients.min()) / (coefficients.max() - coefficients.min())
# 
# # 输出归一化后的结果
# for i, coef in enumerate(normalized_coefficients):
#     print(f"{feature_names[i]} 的归一化系数为: {coef}")

温度 的归一化系数为: 0.26928892545978816
频率 的归一化系数为: 0.12028554683789207
励磁波形 的归一化系数为: 0.25736260170018177
材料 的归一化系数为: 0.2449513056109888
峰值 的归一化系数为: 0.0
温度^2 的归一化系数为: 0.2756596484337104
温度 频率 的归一化系数为: 0.2477872503615341
温度 励磁波形 的归一化系数为: 0.26474249269889244
温度 材料 的归一化系数为: 0.2642632266452732
温度 峰值 的归一化系数为: 0.19789700793965237
频率^2 的归一化系数为: 0.30316344027570435
频率 励磁波形 的归一化系数为: 0.25743905694478064
频率 材料 的归一化系数为: 0.2791265947326045
频率 峰值 的归一化系数为: 1.0
励磁波形^2 的归一化系数为: 0.2685829360018696
励磁波形 材料 的归一化系数为: 0.2692405237906889
励磁波形 峰值 的归一化系数为: 0.2605378003703054
材料^2 的归一化系数为: 0.26526431269984857
材料 峰值 的归一化系数为: 0.3225382084377195
峰值^2 的归一化系数为: 0.5387205325089904


In [58]:
import statsmodels.api as sm

# 根据公式要求计算对数
data['log_P'] = np.log(data['磁芯损耗'])
data['log_f'] = np.log(data['频率'])
data['log_Bm'] = np.log(data['峰值'])
data['log_T'] = np.log(data['温度'])

# 定义自变量 (log(f), log(Bm), log(T), W(励磁波形), M(材料))
X = data[['log_f', 'log_Bm', 'log_T', '励磁波形', '材料']]

# 添加常数项 (k) 到模型中
X = sm.add_constant(X)

# 因变量 (目标变量) 为 log(P)
y = data['log_P']

# 使用最小二乘法拟合模型
model = sm.OLS(y, X).fit()

# 输出模型的总结信息，包括各个参数的估计值
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  log_P   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                 5.802e+04
Date:                Sun, 22 Sep 2024   Prob (F-statistic):               0.00
Time:                        17:32:23   Log-Likelihood:                -5743.3
No. Observations:               12400   AIC:                         1.150e+04
Df Residuals:                   12394   BIC:                         1.154e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4984      0.072    -20.794      0.0

$$\log P = -1.4984 + 1.7011 \log f + 2.3418 \log B_m - 0.4137 \log T + 0.1990 W - 0.0249 M$$